In [20]:
import networkx as nx
import numpy as np

In [52]:
def _BA(size, m):
    G          = nx.barabasi_albert_graph(n=size, m=m)
    # G          = G.to_directed()
    edge_index = np.array(G.edges(), dtype=np.long).T
    edge_w     = np.ones(G.number_of_edges(), dtype=np.float32) * 0.05
    # TODO: random weight
    nx.set_edge_attributes(G, {edge:w for edge, w in zip(G.edges(), edge_w)}, "weight")
    weight_degree = G.degree(weight="weight")
    weight_degree = list(dict(weight_degree).values())
    return edge_index, edge_w, weight_degree, G

In [69]:
e, w, d, G = _BA(100, 5)

In [70]:
with open("ba/attribute.txt", "w") as f:
    f.writelines("n={}\nm={}".format(G.number_of_nodes(), G.number_of_edges()))

In [71]:
with open("ba/graph_ic.inf", "w") as f:
    for edge, weight in zip(e.T, w):
        f.writelines("{} {} {:.3f}\n".format(edge[0], edge[1], weight))

In [61]:
nx.degree(G)

DiDegreeView({0: 40, 1: 114, 2: 78, 3: 120, 4: 86, 5: 42, 6: 58, 7: 46, 8: 56, 9: 40, 10: 82, 11: 34, 12: 48, 13: 34, 14: 16, 15: 50, 16: 20, 17: 34, 18: 100, 19: 22, 20: 20, 21: 18, 22: 12, 23: 36, 24: 30, 25: 40, 26: 56, 27: 54, 28: 30, 29: 28, 30: 40, 31: 16, 32: 18, 33: 22, 34: 10, 35: 24, 36: 24, 37: 34, 38: 16, 39: 22, 40: 20, 41: 16, 42: 16, 43: 12, 44: 18, 45: 14, 46: 10, 47: 12, 48: 18, 49: 14, 50: 10, 51: 20, 52: 32, 53: 14, 54: 8, 55: 12, 56: 20, 57: 10, 58: 18, 59: 18, 60: 8, 61: 20, 62: 10, 63: 14, 64: 16, 65: 12, 66: 30, 67: 26, 68: 26, 69: 12, 70: 10, 71: 12, 72: 22, 73: 12, 74: 14, 75: 24, 76: 12, 77: 34, 78: 12, 79: 10, 80: 12, 81: 12, 82: 16, 83: 14, 84: 22, 85: 24, 86: 16, 87: 14, 88: 12, 89: 8, 90: 30, 91: 8, 92: 14, 93: 6, 94: 6, 95: 18, 96: 10, 97: 10, 98: 14, 99: 6, 100: 10, 101: 20, 102: 6, 103: 8, 104: 8, 105: 22, 106: 6, 107: 20, 108: 14, 109: 14, 110: 10, 111: 10, 112: 6, 113: 10, 114: 10, 115: 6, 116: 8, 117: 14, 118: 16, 119: 12, 120: 16, 121: 8, 122: 20, 1

In [98]:
def _ER(n, p):
    G          = nx.erdos_renyi_graph(n=n, p=p, directed=True)
    # G          = G.to_directed()
    edge_index = np.array(G.edges(), dtype=np.long).T
    edge_w     = np.ones(G.number_of_edges(), dtype=np.float32) * 0.05
    # TODO: random weight
    nx.set_edge_attributes(G, {edge:w for edge, w in zip(G.edges(), edge_w)}, "weight")
    weight_degree = G.degree(weight="weight")
    weight_degree = list(dict(weight_degree).values())
    return edge_index, edge_w, weight_degree, G

In [99]:
e, w, d, G = _ER(500, 0.05)

In [100]:
with open("er/attribute.txt", "w") as f:
    f.writelines("n={}\nm={}".format(G.number_of_nodes(), G.number_of_edges()))

In [101]:
with open("er/graph_ic.inf", "w") as f:
    for edge, weight in zip(e.T, w):
        f.writelines("{} {} {:.3f}\n".format(edge[0], edge[1], weight))

In [102]:
ic = IC(edge_list=e, edge_w=w)

In [106]:
ic.run(np.array([394, 411, 169, 77, 282]))

array(160.3024, dtype=float32)

In [107]:
ic.run(np.array(list(range(100,105))))

array(129.06648, dtype=float32)

In [94]:
import networkx as nx
import torch as T
from torch_scatter import scatter_mul, scatter_add
import pickle as pkl
import time

class IC():
    def __init__(self, edge_list, edge_w, device="cpu", max_iter=10): 
        self.device = device

        self.src_nodes = T.LongTensor(edge_list[0]).to(device)
        self.tar_nodes = T.LongTensor(edge_list[1]).to(device)
        self.weights   = T.FloatTensor(edge_w).to(device)
        self.cave_index = T.LongTensor(self.cave(edge_list)).to(device)
        
        self.N = max([T.max(self.src_nodes), T.max(self.tar_nodes)]).item()+1
        self.E = len(self.src_nodes)
        self.out_weight_d = scatter_add(self.weights, self.src_nodes).to(device)

        self.max_iter = max_iter

    def cave(self, edge_list):
        G = nx.DiGraph()
        edge_list = [(s,t) for s, t in zip(*edge_list)]
        G.add_edges_from(edge_list)
        attr = {edge:w for edge, w in zip(edge_list, range(len(edge_list)))}
        nx.set_edge_attributes(G, attr, "idx")

        cave = []
        for edge in edge_list:
            if G.has_edge(*edge[::-1]):
                cave.append(G.edges[edge[::-1]]["idx"])
            else:
                cave.append(len(edge_list))
        return cave

    def _set_seeds(self, seed_list):
        self.seeds = seed_list if T.is_tensor(seed_list) else T.Tensor(seed_list)
        self.seeds = self.seeds.to(self.device)
        self.Ps_i_0 = 1 - self.seeds

        self.Theta_0 = T.ones(self.E).to(self.device)        # init Theta(t=0)
        self.Ps_0 = 1 - self.seeds[self.src_nodes]    # Ps(t=0)
        self.Phi_0 = 1 - self.Ps_0 # init Thetau(t=0)

        self.Theta_t = self.Theta_0 - self.weights * self.Phi_0 + 1E-10 #get rid of NaN
        self.Ps_t_1 = self.Ps_0             # Ps(t-1)
        self.Ps_t = self.Ps_0 * self.mulmul(self.Theta_t) # Ps(t)
        self.inf_log = [self.seeds.sum(), self.influence()]


    def mulmul(self, Theta_t):
        Theta = scatter_mul(Theta_t, index=self.tar_nodes) # [N]
        Theta = Theta[self.src_nodes] #[E]
        Theta_cav = scatter_mul(Theta_t, index=self.cave_index)[:self.E]

        mul = Theta / Theta_cav
        return mul

    def forward(self):
        Phi_t = self.Ps_t_1 - self.Ps_t
        self.Theta_t = self.Theta_t - self.weights * Phi_t
        Ps_new = self.Ps_0 * self.mulmul(self.Theta_t)

        self.Ps_t_1 = self.Ps_t
        self.Ps_t   = Ps_new
    
    def influence(self):
        # Ps_i : the probability of node i being S 
        self.Ps_i = self.Ps_i_0 * scatter_mul(self.Theta_t, index=self.tar_nodes)
        return T.sum(1-self.Ps_i)
        
    def theta_aggr(self):
        theta = scatter_mul(self.Theta_t, index=self.tar_nodes)

        return theta, self.Ps_i

    def run(self, seed_list):
        seed_list = seed_list.squeeze()
        if len(seed_list) != self.N:
            _seed_list = T.zeros(self.N)
            _seed_list[seed_list] = 1
            seed_list = _seed_list
        self._set_seeds(seed_list)
        for _ in range(self.max_iter):
            self.forward()
            new_inf = self.influence()

            if abs(new_inf - self.inf_log[-1]) < 1.0:
                break
            else:
                self.inf_log.append(new_inf)

        return self.inf_log[-1].numpy()

